# 📘 Feature Engineering для табличных данных

Этот ноутбук — **практический гайд** по feature engineering для табличных данных.

В нём собраны:
- базовые и продвинутые приёмы;
- понятные примеры кода;
- комментарии: *когда* и *зачем* использовать тот или иной приём.

Все примеры написаны так, чтобы ты мог копировать куски кода в свои проекты.

In [ ]:
# Базовые импорты
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

## 🧱 Шаг 0. Создадим примерный датасет

Работать будем на игрушечном датасете интернет-магазина:

In [ ]:
# Пример датафрейма с заказами интернет-магазина
df = pd.DataFrame({
    'user_id':   [101, 101, 102, 103, 103, 103, 104, 105],
    'order_id':  [1,   2,   3,   4,   5,   6,   7,   8],
    'order_amount': [1200, 3000, 500, 7000, 2500, 800, 1500, 400],
    'items_count':  [2,    5,    1,   10,   3,    1,   2,    1],
    'city':     ['Moscow', 'Moscow', 'SPb', 'Kazan', 'Kazan', 'Kazan', 'SPb', 'Moscow'],
    'device':   ['mobile', 'desktop', 'mobile', 'mobile', 'tablet', 'desktop', 'desktop', 'mobile'],
    'order_ts': pd.to_datetime([
        '2024-01-10 10:00',
        '2024-01-15 22:30',
        '2024-01-11 09:10',
        '2024-01-12 18:20',
        '2024-01-20 14:05',
        '2024-01-21 23:55',
        '2024-01-25 12:40',
        '2024-01-26 08:15',
    ]),
    # Таргет: вернули заказ (1) или нет (0)
    'is_returned': [0, 1, 0, 0, 1, 0, 0, 1],
    # Небольшой текстовый комментарий
    'comment': [
        'late payment on last loan',
        'good client, stable income',
        'no credit history',
        'late late payment',
        'long time customer',
        'delayed payment again',
        'regular customer',
        'bad experience, late delivery'
    ]
})

df

## 🟩 Шаг 1. Фичи из даты и времени

**Когда использовать:**
- есть временная метка события (покупка, клик, измерение сенсора);
- поведение людей/системы может зависеть от времени.

**Зачем:**
- учесть сезонность, ночные/дневные паттерны, выходные, плотность активности.

In [ ]:
# Разбор даты/времени на компоненты
# Здесь order_ts — datetime-колонка

# Год, месяц, день, час, день недели
df['year'] = df['order_ts'].dt.year             # долгосрочные тренды
df['month'] = df['order_ts'].dt.month           # сезонность
df['day'] = df['order_ts'].dt.day
df['hour'] = df['order_ts'].dt.hour             # рабочие/ночные часы
df['weekday'] = df['order_ts'].dt.weekday       # Monday=0, Sunday=6

# Флаг выходного (часто другое поведение пользователей)
df['is_weekend'] = df['weekday'].isin([5, 6]).astype(int)

# Флаг ночного заказа (импульсивные покупки, высокий риск возврата)
df['is_night'] = ((df['hour'] >= 22) | (df['hour'] <= 6)).astype(int)

# Цикличные признаки для месяца (чтобы 12 и 1 были "рядом")
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

df[['order_ts', 'year', 'month', 'day', 'hour', 'weekday', 'is_weekend', 'is_night']].head()

In [ ]:
# Время с предыдущего заказа пользователя
# Когда использовать: есть user_id и серия событий во времени
# Зачем: понять, как часто клиент делает заказы (частота = вовлечённость)

df = df.sort_values(['user_id', 'order_ts'])
df['prev_ts'] = df.groupby('user_id')['order_ts'].shift(1)
df['time_since_prev'] = (df['order_ts'] - df['prev_ts']).dt.total_seconds()

# Первому событию пользователя нечем сравнивать — заполним медианой
df['time_since_prev'] = df['time_since_prev'].fillna(df['time_since_prev'].median())

df[['user_id', 'order_ts', 'prev_ts', 'time_since_prev']]

## 🟨 Шаг 2. Числовые фичи: логарифмы, отношения, бининги

**Когда:**
- числовые признаки с перекошенным распределением (длинный хвост);
- важны пропорции, а не сами значения (loan/income, price/salary).

**Зачем:**
- сгладить выбросы;
- дать модели более осмысленные масштабы.

In [ ]:
# Логарифмирование — уменьшение влияния больших значений
df['order_amount_log'] = np.log1p(df['order_amount'])

# Отношение: средняя цена за товар
df['amount_per_item'] = df['order_amount'] / (df['items_count'] + 1e-6)

# Ранжирование по величине чека (процентиль)
df['order_amount_rank'] = df['order_amount'].rank(pct=True)

# Биннинг возраста по квантилям (пример с искусственным age)
df['age'] = [25, 26, 40, 41, 60, 61, 30, 35]  # просто для примера
df['age_bin'] = pd.qcut(df['age'], 4, labels=False)

df[['order_amount', 'order_amount_log', 'amount_per_item', 'order_amount_rank', 'age', 'age_bin']]

## 🟦 Шаг 3. Категориальные фичи: One-Hot и частоты

**Когда:**
- есть признаки типа город, устройство, тип клиента.

**Зачем:**
- модели работают с числами;
- частота категории хорошо описывает её "важность".

In [ ]:
# One-Hot Encoding через pandas
# Когда: категорий мало, модель линейная/нейросеть
df_ohe = pd.get_dummies(df, columns=['city', 'device'], prefix=['city', 'dev'])

df_ohe.head()

In [ ]:
# Частота категории (frequency encoding)
# Когда: категорий много, а ID-like признаки

df['city_freq'] = df['city'].map(df['city'].value_counts())
df[['city', 'city_freq']].drop_duplicates()

## 🟥 Шаг 4. Агрегации по user_id (groupby)

**Что такое агрегация:**
- объединяем несколько строк в одну *по группе* (например, по пользователю);
- считаем среднее, сумму, количество, максимум и т.д.

**Когда использовать:**
- когда `user_id`, `product_id`, `shop_id` встречаются много раз.

**Зачем:**
- описать *поведение в целом*, а не только один заказ.

In [ ]:
# Считаем агрегаты по пользователю
user_stats = df.groupby('user_id')['order_amount'].agg(
    user_order_mean='mean',   # средний чек
    user_order_sum='sum',     # суммарные траты
    user_order_count='count'  # количество заказов
).reset_index()

user_stats

In [ ]:
# Мержим агрегаты обратно в основной датафрейм
df = df.merge(user_stats, on='user_id', how='left')

# Девиации и отношения относительно среднего по пользователю
df['amount_dev_from_user_mean'] = df['order_amount'] - df['user_order_mean']
df['amount_ratio_to_user_mean'] = df['order_amount'] / (df['user_order_mean'] + 1e-6)

df[['user_id', 'order_amount', 'user_order_mean', 'user_order_sum', 'user_order_count',
    'amount_dev_from_user_mean', 'amount_ratio_to_user_mean']]

## 🟪 Шаг 5. Бинарные фичи (флаги по условиям)

**Когда:**
- есть пороги в бизнес-логике (большой чек, много товаров);
- нужно явно выделить особые случаи.

**Зачем:**
- линейные модели плохо видят пороги;
- флаги часто ловят аномалии и риски.

In [ ]:
# Примеры фичей по условию

# np.where для простого if-else
df['is_big_order'] = np.where(df['order_amount'] > df['order_amount'].median(), 1, 0)

# .isin() для списка значений
df['is_big_city'] = df['city'].isin(['Moscow', 'SPb']).astype(int)

# Комбинация условий (ночной и дорогой заказ)
df['is_night_and_big'] = (
    (df['is_night'] == 1) &
    (df['is_big_order'] == 1)
).astype(int)

df[['order_amount', 'is_big_order', 'city', 'is_big_city', 'is_night', 'is_night_and_big']]

## 🟫 Шаг 6. Time Series фичи: лаги и скользящие окна

**Когда:**
- есть временной ряд или события по времени, где важен порядок.

**Зачем:**
- дать модели "память" о предыдущих значениях.

In [ ]:
# Лаговые признаки (предыдущее значение по пользователю)
df = df.sort_values(['user_id', 'order_ts'])
df['order_amount_lag1'] = df.groupby('user_id')['order_amount'].shift(1)

# Скользящее среднее по окну из 2 заказов
df['order_amount_roll_mean2'] = (
    df.groupby('user_id')['order_amount']
      .rolling(window=2)
      .mean()
      .reset_index(level=0, drop=True)
)

df[['user_id', 'order_ts', 'order_amount', 'order_amount_lag1', 'order_amount_roll_mean2']]

## 🟧 Шаг 7. Простые текстовые фичи

**Когда:**
- есть короткие текстовые поля (комментарии, описания).

**Зачем:**
- выжать полезную инфу без heavy NLP.

In [ ]:
# Простые признаки из текстового поля comment

df['comment_len'] = df['comment'].str.len()
df['comment_word_count'] = df['comment'].str.split().str.len()
df['has_late'] = df['comment'].str.contains('late', case=False).astype(int)
df['has_bad'] = df['comment'].str.contains('bad|poor', case=False).astype(int)

df[['comment', 'comment_len', 'comment_word_count', 'has_late', 'has_bad']]

## 🟥 Шаг 8. Target Encoding (аккуратно, без утечки)

**Когда:**
- категориальных значений много, а связь с таргетом важна (например, город, товар).

**Зачем:**
- заменить категорию числом, отражающим средний таргет для неё.

⚠️ Важно: target encoding нужно делать **внутри KFold** на train, чтобы не подсмотреть target.

In [ ]:
# Пример target encoding для колонки city
target_col = 'is_returned'
col = 'city'

df_te = df.copy()
df_te[col + '_te'] = np.nan

kf = KFold(n_splits=3, shuffle=True, random_state=42)

for tr_idx, val_idx in kf.split(df_te):
    train_fold = df_te.iloc[tr_idx]
    val_fold = df_te.iloc[val_idx]

    means = train_fold.groupby(col)[target_col].mean()
    df_te.loc[val_idx, col + '_te'] = df_te.loc[val_idx, col].map(means)

# Заполняем редкие/новые категории глобальным средним
global_mean = df_te[target_col].mean()
df_te[col + '_te'] = df_te[col + '_te'].fillna(global_mean)

df_te[[col, target_col, col + '_te']].drop_duplicates().sort_values(col)

## 🟨 Шаг 9. Добавление колонок по условию + пример с "средним возрастом выживших"

Иногда нужно создать признак на основе глобальной или групповой статистики.

Например: *средний возраст выживших* в задаче Titanic.

In [ ]:
# Маленький пример в стиле Titanic

titanic = pd.DataFrame({
    'Survived': [1, 0, 1, 0, 1, 0],
    'Age':      [22, 38, 26, 35, 18, 54],
    'Sex':      ['male', 'female', 'female', 'female', 'male', 'male']
})

# 1) Глобальный средний возраст среди выживших
mean_age_survived = titanic.loc[titanic['Survived'] == 1, 'Age'].mean()
titanic['mean_age_survived_global'] = mean_age_survived

# 2) Средний возраст выживших по полу
mean_age_by_sex = titanic[titanic['Survived'] == 1].groupby('Sex')['Age'].mean()
titanic['mean_age_survived_by_sex'] = titanic['Sex'].map(mean_age_by_sex)

# 3) Девиация возраста от среднего возраста выживших того же пола
titanic['age_dev_from_survivors'] = titanic['Age'] - titanic['mean_age_survived_by_sex']

titanic

## ✅ Резюме

В этом ноутбуке ты увидел основные блоки feature engineering для табличных данных:

- фичи из даты и времени;
- числовые преобразования (логарифмы, отношения, ранги, биннинг);
- работа с категориальными признаками (One-Hot, частотное кодирование);
- агрегаты по группам (user_id и пр.) + девиации;
- бинарные фичи по условиям (флаги);
- временные ряды: лаги и скользящие окна;
- простые текстовые фичи;
- аккуратный target encoding без утечки;
- пример со "средним возрастом выживших".

Дальше ты можешь:
- копировать нужные блоки прямо в свой код;
- адаптировать имена колонок под свои данные;
- накидывать новые признаки по той же логике: *что в реальной жизни влияет на таргет?*

Если захочешь, можно отдельным ноутбуком сделать **гайд по отбору фичей** (SHAP, Permutation Importance, корреляции).